In [2]:
import data
similarity_standard_dir = './wordsim353_sim_rel/wordsim_similarity_goldstandard.txt'
data_dir='./data/wikitext-2'
corpus = data.Corpus(data_dir)

In [14]:
with open(similarity_standard_dir, 'r') as file:
    words1 = []
    words1_idx = []
    words2 = []
    words2_idx = []
    scores = []
    for line in file:
        line = line.rstrip('\n')
        seq = line.split('\t')
        word1, word2, score = seq[0], seq[1], seq[2]
        score = float(score)
        if word1 in list(corpus.dictionary.word2idx.keys()) and word2 in list(corpus.dictionary.word2idx.keys()):
            words1.append(word1)
            words2.append(word2)
            words1_idx.append(corpus.dictionary.word2idx[word1])
            words2_idx.append(corpus.dictionary.word2idx[word2])
            scores.append(score)

In [17]:
import pandas as pd
df = pd.DataFrame({'word1': words1, 'word2': words2, 'score': scores})
df.head(10)

,word1,word2,score
0,tiger,cat,7.35
1,tiger,tiger,10.00
2,plane,car,5.77
3,train,car,6.31
4,television,radio,6.77
5,media,radio,7.42
6,bread,butter,6.19
7,doctor,nurse,7.00
8,professor,doctor,6.62
9,student,professor,6.81


In [25]:
import torch
from TrainFNN import FNNModel
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
model = torch.load('model.pt', map_location=device)
model.eval()
idx1 = torch.tensor(words1_idx).to(device)
idx2 = torch.tensor(words2_idx).to(device)

In [30]:
with torch.no_grad():
    emb1 = model.encoder(idx1).detach().cpu()
    emb2 = model.encoder(idx2).detach().cpu()
cos_score = torch.nn.functional.cosine_similarity(emb1, emb2)

In [31]:
from scipy.stats import spearmanr
spearmanr(scores, cos_score)

SpearmanrResult(correlation=0.06408710638960426, pvalue=0.3913830419460702)

In [33]:
from TrainWithDropOut import FNNModelDropout
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
model = torch.load('model_dropout.pt', map_location=device)
model.eval()
with torch.no_grad():
    emb1 = model.encoder(idx1).detach().cpu()
    emb2 = model.encoder(idx2).detach().cpu()
cos_score = torch.nn.functional.cosine_similarity(emb1, emb2)
spearmanr(scores, cos_score)

SpearmanrResult(correlation=0.09964203233707374, pvalue=0.18201294598775636)

In [34]:
from TrainWithSharingWeights import FNNModelSharingWeights
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
model = torch.load('model_dropout_SharingWeights.pt', map_location=device)
model.eval()
with torch.no_grad():
    emb1 = model.encoder(idx1).detach().cpu()
    emb2 = model.encoder(idx2).detach().cpu()
cos_score = torch.nn.functional.cosine_similarity(emb1, emb2)
spearmanr(scores, cos_score)

SpearmanrResult(correlation=0.30001694998741, pvalue=4.073378209394652e-05)